In [74]:
import time
import random
import pickle
import threading
import seal
from seal import ChooserEvaluator, \
    Ciphertext, \
    Decryptor, \
    Encryptor, \
    EncryptionParameters, \
    Evaluator, \
    IntegerEncoder, \
    FractionalEncoder, \
    KeyGenerator, \
    MemoryPoolHandle, \
    Plaintext, \
    SEALContext, \
    EvaluationKeys, \
    GaloisKeys, \
    PolyCRTBuilder, \
    ChooserEncoder, \
    ChooserEvaluator, \
    ChooserPoly

## Init encryption environment

In [75]:
parms = EncryptionParameters()
parms.set_poly_modulus("1x^2048 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(2048))
parms.set_plain_modulus(1 << 8)

context = SEALContext(parms)
print_parameters(context)

keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

/ Encryption parameters:
| poly_modulus: 1x^2048 + 1
| coeff_modulus_size: 56 bits
| plain_modulus: 256
| noise_standard_deviation: 3.19


In [76]:
encoder = FractionalEncoder(context.plain_modulus(), context.poly_modulus(), 64, 32, 3)

## Some basic util functions 

In [77]:
def print_parameters(context):
    print("/ Encryption parameters:")
    print("| poly_modulus: " + context.poly_modulus().to_string())

    # Print the size of the true (product) coefficient modulus
    print("| coeff_modulus_size: " + (str)(context.total_coeff_modulus().significant_bit_count()) + " bits")

    print("| plain_modulus: " + (str)(context.plain_modulus().value()))
    print("| noise_standard_deviation: " + (str)(context.noise_standard_deviation()))

In [78]:
def encode_rationals(numbers, encoder):
    # encoding without encryption
    encoded_coefficients = []
    for i in range(len(coefficients)):
        encoded_coefficients.append(encoder.encode(coefficients[i]))
    return encoded_coefficients

In [79]:
def encode_num(num, encoder):
    # encoding without encryption
    return encoder.encode(num)

In [80]:
def sum_enc_array(array, evaluator):
    encrypted_result = Ciphertext()
    evaluator.add_many(array, encrypted_result)
    return encrypted_result

In [81]:
def decode(encrypted_res, decryptor, encoder):
    plain_result = Plaintext()
    decryptor.decrypt(encrypted_res, plain_result)
    result = encoder.decode(plain_result)
    return result

In [82]:
def encrypt_rationals(rational_numbers, encryptor, encoder):
    # encrypt array of rational numbers

    encrypted_rationals = []
    for i in range(len(rational_numbers)):
        encrypted_rationals.append(Ciphertext(parms))
        encryptor.encrypt(encoder.encode(rational_numbers[i]), encrypted_rationals[i])
    return encrypted_rationals

In [83]:
def weighted_average(encrypted_rationals, encoded_coefficients, encoded_divide_by, evaluator):

    for i in range(len(encrypted_rationals)):
        evaluator.multiply_plain(encrypted_rationals[i], encoded_coefficients[i])
    
    encrypted_result = Ciphertext()
    evaluator.add_many(encrypted_rationals, encrypted_result)
    evaluator.multiply_plain(encrypted_result, encoded_divide_by)

    # How much noise budget do we have left?
    print("Noise budget in result: " + (str)(decryptor.invariant_noise_budget(encrypted_result)) + " bits")

    return encrypted_result

In [88]:
coefficients = [0.1, 0.05, 0.05, 0.2, 0.05, 0.3, 0.1, 0.025, 0.075, 0.05]
rational_numbers = [3.1, 4.159, 2.65, 3.5897, 9.3, 2.3, 8.46, 2.64, 3.383, 2.795]

encrypted_rationals = encrypt_rationals(rational_numbers, encryptor, encoder)
encoded_coefficients = encode_rationals(coefficients, encoder)
divide_by = encode_num(1/len(rational_numbers), encoder)

avg = weighted_average(encrypted_rationals, encoded_coefficients, divide_by, evaluator)

decode(avg, decryptor, encoder)

Noise budget in result: 30 bits


0.3828864999999884

In [90]:
numbers = [3, 4.159, 2.65, 2, -9.3]

enc_all_nums = encrypt_rationals(numbers, encryptor, encoder)

In [93]:
enc_sum1 = sum_enc_array(enc_all_nums, evaluator)
decode(enc_sum1, decryptor, encoder)

2.5089999999999986

In [94]:
def enc_substract(a, b, encoder, evaluator):
    # a and b should be encrypted
    minus_sign = encoder.encode(-1)
    evaluator.multiply_plain(b, minus_sign)
    evaluator.add(a, b)
    return a

In [95]:
def encrypt_num(value, encoder, encryptor):
    plain = encoder.encode(value)
    encrypted = Ciphertext()
    encryptor.encrypt(plain, encrypted)
    return encrypted

In [101]:
a = encrypt_num(5, encoder, encryptor)
b = encrypt_num(3, encoder, encryptor)
res = enc_substract(a, b, encoder, evaluator)
decode(res, decryptor, encoder)

2.0